# Настройка среды

In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))

In [2]:
from belashovplot import TiledPlot
from utilities import *
from utilities.filters import Gaussian, Window
from elements.abstracts import AbstractModulator
from elements.modulators import Lens, PhaseModulator, AmplitudeModulator
from elements.propagators import FurrierPropagation, ConvolutionalPropagation
from elements.composition import CompositeModel, HybridModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from elements.detectors import ClassificationDetectors, MatrixDetectors
from parameters import FigureWidthHeight, FontLibrary
from tqdm import tqdm
from math import sin, sqrt
import torch
import numpy
import timm
import pandas
from copy import deepcopy
from cluster import train, confusion, execute, epochs

In [3]:
FontLibrary.Fonts.PlotTitle.FontSize = 16

FontLibrary.Fonts.DescriptionLeft.FontSize = 12
FontLibrary.Fonts.DescriptionLeft.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionLeft.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionBottom.FontSize = 10
FontLibrary.Fonts.DescriptionBottom.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionBottom.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionTop.FontSize = 10
FontLibrary.Fonts.DescriptionTop.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionTop.FontStyle = 'italic'

FontLibrary.Fonts.ColumnDescriptionTop.FontSize = 9
FontLibrary.Fonts.ColumnDescriptionBottom.FontSize = 9
FontLibrary.Fonts.RowDescriptionLeft.FontSize = 9
FontLibrary.Fonts.RowDescriptionRight.FontSize = 9
FontLibrary.Fonts.AxisX.FontSize = 8
FontLibrary.Fonts.AxisY.FontSize = 8
FigureWidthHeight = (6.69291, 10.1181-1.0)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Используемый девайс:', torch.cuda.get_device_name(device) if torch.cuda.is_available() else 'ЦП')

Используемый девайс: NVIDIA A100-SXM4-80GB


# Определение параметров модели

In [5]:
def resnet18(classes:int=10):
    return timm.create_model('resnet18', pretrained=False,  in_chans=1, num_classes=classes).to(device)

In [6]:
def combined_loss(cross_entropy_to_mse_proportion:float=1.0):
    def loss_function(outputs, targets):
        CELoss = torch.nn.functional.cross_entropy(outputs, targets)
        MSELoss = torch.nn.functional.mse_loss(torch.nn.functional.softmax(outputs, dim=1), torch.nn.functional.one_hot(targets, num_classes=10).float())
        loss = cross_entropy_to_mse_proportion*CELoss + (1.0-cross_entropy_to_mse_proportion)*MSELoss
        return loss
    return loss_function

In [7]:
# Предпочтительные параметры
size = 30.0E-6
near_N = 2004 #1336
near_length = 10.0E-3
wavelength = 500.0E-9
detectors_amount = 24
masks_amount = 5
distance = 0.09052

# Параметры обучения
batch_size = 64
learning_rate = 0.009854
loss_function_proportion = 0.609798
optimizer_type_name = 'RMSprop'
optimizer_types_list = {'Adam':torch.optim.Adam, 'SGD':torch.optim.SGD, 'RMSprop':torch.optim.RMSprop, 'Adagrad':torch.optim.Adagrad}

# Параметры когерентности
spatial_coherence = 100.0E-6
time_coherence = 10.0E-9
time_relaxation = 1.0E-6
mean_samples = 1000

# Вычисляемые параметры
pixels = upper_integer(near_length/size)
length = pixels * size
cppp = upper_integer(near_N * size / length)
N = upper_integer(length*cppp / size) 
detector_size = length / 60

print(f"Длинна волны:                             {engineering(wavelength, 'м')}")
print(f"Количество вычислительных пикселей:       {N}")
print(f"Количество пикселей маски:                {pixels}")
print(f"Размер оптических элементов:              {engineering(length, 'м')}")
print(f"Размер пикселя маски:                     {engineering(length/pixels, 'м')}")
print(f"Расстояние между слоями:                  {engineering(distance, 'м')}")

print(f"Временная когерентность:                  {engineering(time_coherence, 'с')}")
print(f"Время релаксации:                         {engineering(time_relaxation, 'c')}")
print(f"Пространственная когерентность:           {engineering(spatial_coherence, 'м')}")
print(f"Количество усреднений:                    {mean_samples}")

print(f"Размер детекторов:                        {engineering(detector_size, 'м')}")
print(f"Количество детекторов:                    {detectors_amount} на {detectors_amount}")

print(f"Пропорция CE к MSE лосс:                  {loss_function_proportion}")
print(f"Размер батча:                             {batch_size}")
print(f"Тип оптимизатора:                         {optimizer_type_name}")

Длинна волны:                             500.0 нм
Количество вычислительных пикселей:       2004
Количество пикселей маски:                334
Размер оптических элементов:              10.02 мм
Размер пикселя маски:                     30.0 мкм
Расстояние между слоями:                  90.52 мм
Временная когерентность:                  10.0 нс
Время релаксации:                         1.0 мкc
Пространственная когерентность:           100.0 мкм
Количество усреднений:                    1000
Размер детекторов:                        167.0 мкм
Количество детекторов:                    24 на 24
Пропорция CE к MSE лосс:                  0.609798
Размер батча:                             64
Тип оптимизатора:                         RMSprop


# Инициализация моделей

In [8]:
# Модуль некогерентности
incoherent = Incoherent(spatial_coherence, time_coherence, time_relaxation, mean_samples, N, length).to(device)
incoherent_encoder, incoherent_decoder = incoherent.pair()
incoherent_encoder, incoherent_decoder = incoherent_encoder.to(device), incoherent_decoder.to(device)

# Модули детекторов
spectral_filter = Window(centers=wavelength, sizes=300.0E-9)
detectors_filter = Gaussian((detector_size, detector_size), (0,0))
detectors_incoherent = MatrixDetectors(N, length, wavelength, detectors_amount, detectors_filter, spectral_filter).to(device)
detectors_coherent = deepcopy(detectors_incoherent)

# Электронные модели
electronic_incoherent = resnet18()
electronic_coherent = deepcopy(electronic_incoherent)

# Оптические модели
propagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.4)
phase_modulators = [PhaseModulator(N, length, pixels) for i in range(masks_amount)]
amplitude_modulators = [AmplitudeModulator(N, length, pixels) for i in range(masks_amount)]
elements = [phase_modulators[0], amplitude_modulators[0]]
for phase_modulator, amplitude_modulator in zip(phase_modulators[1:], amplitude_modulators[1:]):
    elements.append(propagation)
    elements.append(phase_modulator)
    elements.append(amplitude_modulator)
elements.append(propagation)
optical_incoherent = CompositeModel(incoherent_encoder, *deepcopy(elements), incoherent_decoder).to(device)
optical_coherent = CompositeModel(*deepcopy(elements)).to(device)

# Гибридные модели
hybrid_model_incoherent = HybridModel(optical_incoherent, detectors_incoherent, electronic_incoherent).to(device)
hybrid_model_coherent = HybridModel(optical_coherent, detectors_coherent, electronic_coherent).to(device)

# Распределённые модели
model_incoherent = hybrid_model_incoherent
model_coherent = hybrid_model_coherent

# Набор данных
dataset = Dataset('CIFAR10', batch_size, N, N, torch.complex64)
dataset.train
dataset.test

# Тип оптимизатора и лосс-функция
loss_function = combined_loss(loss_function_proportion)
optimizer_type = optimizer_types_list[optimizer_type_name]

# Обучение модели

In [9]:
# models_history, loss_histories, confusion_matrices_history = epochs(10, 10, model_incoherent, dataset, torch.nn.CrossEntropyLoss(), torch.optim.Adam, lr=0.0003)

In [10]:
# models_history1, loss_histories1, confusion_matrices_history1 = epochs(10, 10, model_coherent, dataset, torch.nn.CrossEntropyLoss(), torch.optim.Adam, lr=0.0003)

In [11]:
models_history, loss_histories, confusion_matrices_history = epochs(20, 10, model_incoherent, dataset, torch.nn.CrossEntropyLoss(), torch.optim.Adam, lr=0.0003)

Training thread#0 PID is: 1721377
IncoherentEncoder
PhaseModulator
AmplitudeModulator
FurrierPropagation
PhaseModulator
AmplitudeModulator
FurrierPropagation
PhaseModulator
AmplitudeModulator
FurrierPropagation
PhaseModulator
AmplitudeModulator
FurrierPropagation
PhaseModulator
AmplitudeModulator
FurrierPropagation
IncoherentDecoder
MatrixDetectors
ResNet
Training thread#1 PID is: 1721448
Training thread#2 PID is: 1721648
Training thread#3 PID is: 1721848
Training thread#4 PID is: 1722047
Training thread#5 PID is: 1722251
Training thread#6 PID is: 1722450
Training thread#7 PID is: 1722650
100%|███████████████████████████████████████████| 98/98 [02:47<00:00,  1.71s/it]
Accuracy in the beggining is 10.0
RLoss: 1.7902157069641185, RPI1000: -6.362468125640822: 100%|█| 98/98 [12:46<00:
100%|███████████████████████████████████████████| 98/98 [02:41<00:00,  1.65s/it]
Accuracy after epoch 1 is 13.662
RLoss: 1.610902886012226, RPI1000: -1.918593511962113: 100%|█| 98/98 [12:55<00:0
100%|████████

ValueError: Default process group has not been initialized, please make sure to call init_process_group.

In [ ]:
# models_history1, loss_histories1, confusion_matrices_history1 = epochs(30, 10, model_coherent, dataset, torch.nn.CrossEntropyLoss(), torch.optim.Adam, lr=0.0003)